# Дослідження даних Life Expectancy та побудова моделей

У цьому ноутбуку:
1. Перевіряємо дослідницькі гіпотези
2. Будуємо моделі прогнозування тривалості життя
3. Порівнюємо різні алгоритми ML

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from data_load import load_data
from data_research import (
    prepare_data_for_modeling,
    train_linear_regression,
    train_random_forest,
    train_gradient_boosting,
    compare_models,
    get_feature_importance,
    calculate_correlation_with_target
)

plt.style.use('seaborn-v0_8')
%matplotlib inline

# Налаштування відображення
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Завантаження та огляд даних

In [ ]:
df = load_data()
print(f"Розмір: {df.shape}")
df.head()

In [ ]:
# Базова статистика
df.describe()

## 2. Гіпотеза 1: Вплив ВВП на тривалість життя

**Питання:** Чи існує кореляція між ВВП країни та очікуваною тривалістю життя?

**Гіпотеза:** Країни з вищим ВВП мають вищу очікувану тривалість життя.

In [ ]:
# Аналіз кореляції GDP vs Life Expectancy
gdp_col = 'GDP'
life_col = 'Life expectancy '

# Видаляємо пропущені значення
df_clean = df[[gdp_col, life_col]].dropna()

# Кореляція Пірсона
correlation, p_value = stats.pearsonr(df_clean[gdp_col], df_clean[life_col])

print(f"Кореляція Пірсона: {correlation:.4f}")
print(f"P-value: {p_value:.4e}")
print(f"\nВисновок: {'Значуща' if p_value < 0.05 else 'Незначуща'} кореляція")

# Візуалізація
plt.figure(figsize=(10, 6))
plt.scatter(df_clean[gdp_col], df_clean[life_col], alpha=0.5)
plt.xlabel('GDP')
plt.ylabel('Life Expectancy (років)')
plt.title(f'GDP vs Life Expectancy (r={correlation:.3f}, p={p_value:.2e})')

# Лінія тренду
z = np.polyfit(df_clean[gdp_col], df_clean[life_col], 1)
p = np.poly1d(z)
plt.plot(df_clean[gdp_col], p(df_clean[gdp_col]), "r--", alpha=0.8, label='Trend line')
plt.legend()
plt.tight_layout()
plt.show()

## 3. Гіпотеза 2: Ефективність імунізації

**Питання:** Як рівень імунізації впливає на смертність дітей?

**Гіпотеза:** Високий рівень імунізації знижує дитячу смертність.

In [ ]:
# Середній рівень імунізації (поліомієліт, дифтерія, гепатит B)
immunization_cols = ['Polio', 'Diphtheria ', 'Hepatitis B']
mortality_col = 'under-five deaths '

# Створюємо середній показник імунізації
df_immune = df[immunization_cols + [mortality_col]].dropna()
df_immune['Avg_Immunization'] = df_immune[immunization_cols].mean(axis=1)

# Кореляція
correlation, p_value = stats.pearsonr(
    df_immune['Avg_Immunization'], 
    df_immune[mortality_col]
)

print(f"Кореляція (Імунізація vs Дитяча смертність): {correlation:.4f}")
print(f"P-value: {p_value:.4e}")
print(f"\nВисновок: {'Підтверджено' if correlation < 0 and p_value < 0.05 else 'Не підтверджено'}")

# Візуалізація
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Scatter plot
axes[0].scatter(df_immune['Avg_Immunization'], df_immune[mortality_col], alpha=0.5)
axes[0].set_xlabel('Середній рівень імунізації (%)')
axes[0].set_ylabel('Смертність дітей до 5 років')
axes[0].set_title(f'Імунізація vs Дитяча смертність (r={correlation:.3f})')

# Box plot - групи за рівнем імунізації
df_immune['Immunization_Group'] = pd.cut(
    df_immune['Avg_Immunization'], 
    bins=[0, 60, 80, 100],
    labels=['Низький (<60%)', 'Середній (60-80%)', 'Високий (>80%)']
)
df_immune.boxplot(column=mortality_col, by='Immunization_Group', ax=axes[1])
axes[1].set_xlabel('Рівень імунізації')
axes[1].set_ylabel('Смертність дітей до 5 років')
axes[1].set_title('Дитяча смертність за рівнем імунізації')
plt.suptitle('')

plt.tight_layout()
plt.show()

## 4. Гіпотеза 3: Багатофакторна модель прогнозування

**Питання:** Чи можна створити точну ML модель для прогнозування тривалості життя?

**Підхід:** Використовуємо різні алгоритми і порівнюємо їх ефективність.

In [ ]:
# Підготовка даних
X_train, X_test, y_train, y_test, features = prepare_data_for_modeling(df)

print(f"Train set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"\nFeatures ({len(features)}): {features[:10]}...")

In [ ]:
# Навчання різних моделей
print("Навчання Linear Regression...")
lr_results = train_linear_regression(X_train, y_train, X_test, y_test)

print("Навчання Random Forest...")
rf_results = train_random_forest(X_train, y_train, X_test, y_test, n_estimators=100)

print("Навчання Gradient Boosting...")
gb_results = train_gradient_boosting(X_train, y_train, X_test, y_test, n_estimators=100)

print("✓ Моделі навчено")

In [ ]:
# Порівняння моделей
comparison = compare_models({
    'Linear Regression': lr_results,
    'Random Forest': rf_results,
    'Gradient Boosting': gb_results
})

print("\n" + "="*70)
print("ПОРІВНЯННЯ МОДЕЛЕЙ")
print("="*70)
display(comparison)

# Знаходимо найкращу модель
best_model = comparison.loc[comparison['Test R²'].idxmax(), 'Model']
best_r2 = comparison['Test R²'].max()
print(f"\n🏆 Найкраща модель: {best_model} (R² = {best_r2:.4f})")

In [ ]:
# Візуалізація прогнозів найкращої моделі
best_results = rf_results  # Зазвичай Random Forest показує кращі результати

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Train predictions
axes[0].scatter(y_train, best_results['predictions']['y_train_pred'], alpha=0.5)
axes[0].plot([y_train.min(), y_train.max()], 
             [y_train.min(), y_train.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Life Expectancy')
axes[0].set_ylabel('Predicted Life Expectancy')
axes[0].set_title(f"Train Set (R² = {best_results['train_metrics']['r2']:.4f})")

# Test predictions
axes[1].scatter(y_test, best_results['predictions']['y_test_pred'], alpha=0.5)
axes[1].plot([y_test.min(), y_test.max()], 
             [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1].set_xlabel('Actual Life Expectancy')
axes[1].set_ylabel('Predicted Life Expectancy')
axes[1].set_title(f"Test Set (R² = {best_results['test_metrics']['r2']:.4f})")

plt.tight_layout()
plt.show()

## 5. Важливість факторів

In [ ]:
# Feature importance для Random Forest
importance_df = get_feature_importance(rf_results, top_n=15)

plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.xlabel('Важливість')
plt.title('Топ-15 найважливіших факторів (Random Forest)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nТоп-10 найважливіших факторів:")
display(importance_df.head(10))

In [ ]:
# Кореляції з Life Expectancy
correlations = calculate_correlation_with_target(df, target='Life expectancy ', top_n=15)

plt.figure(figsize=(10, 6))
colors = ['green' if x > 0 else 'red' for x in correlations['Correlation']]
plt.barh(correlations['Feature'], correlations['Correlation'], color=colors)
plt.xlabel('Кореляція')
plt.title('Топ-15 факторів за кореляцією з Life Expectancy')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nКореляції:")
display(correlations)

## 6. Висновки

### Підтвердження гіпотез:

1. **Гіпотеза 1 (GDP)**: Результати вказують на кореляцію між ВВП та тривалістю життя

2. **Гіпотеза 2 (Імунізація)**: Рівень імунізації корелює з дитячою смертністю

3. **Гіпотеза 3 (ML модель)**: Побудовано моделі з високою точністю прогнозування

### Найважливіші фактори:
- HIV/AIDS
- Income composition of resources
- Schooling
- Adult Mortality
- GDP